This is a type of location optimization analysis, specifically finding the optimal location of facilites on a network. This analysis is the P-Median Problem implemented in **Julia**:

### Capacitated P-Median Problem
The P-median problem finds the location of (a pre-specified number of) P facilities to minimize the average travel distance (or time) among all demand points and facilities. The P-median problem can take into account the level of demand at each point (e.g. number of people, or the number of visits). The capacitated version of the P-median problem adds capacity contraints to facilities, so that customers may be directed to the nearest available facility if capacity is reached.

more information on GOSTNets Optimization can be found in the wiki: https://github.com/worldbank/GOST_PublicGoods/wiki/GOSTnets-Optimization

#### This is a Julia Notebook. If you are new to Julia, these are the [steps](https://datatofish.com/add-julia-to-jupyter/) to add Julia to a Jupyter Notebook

In [1]:
using Pkg
Pkg.add("JuMP")
Pkg.add("Cbc")
Pkg.add("MathOptInterface")
Pkg.add("MathProgBase")
Pkg.add("CSV")
Pkg.add("DelimitedFiles")
Pkg.add("DataFrames")
println("Done installing packages")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %.1 %          ]  72.6 % Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/enviro

In [2]:
#using JuMP, Cbc, GLPK, CPLEX, Test, Random, MathOptInterface, MathOptFormat, CSV, DataFrames, DelimitedFiles, MathProgBase
using JuMP, Cbc, MathOptInterface, CSV, DataFrames, DelimitedFiles, MathProgBase

In [3]:
# MathOptInterface is an abstraction layer for mathematical optimization solvers
const MOI = MathOptInterface

MathOptInterface

## This is the Julia Capacitated P-Median function

In [110]:
function Cap_pMedian(numFacility::Int, CSVfile, origins_pop_dict, facilities_cap_dict)

    println("numFacility")
    println(numFacility)

    # materialize a csv file as a DataFrame
    df = CSV.File(CSVfile) |> DataFrame!

    #extract column_headers
    column_headers = []
    #skip Column1
    for i=2:length(names(df))
      push!(column_headers,String(names(df)[i]))
    end
    
    OD_dict = Dict()
    for i in 1:size(df, 1)
        OD_dict[df[i,1]] = df[i,2:end]
    end

    #println("print OD_dict")
    #println(OD_dict)

    #origins as array
    origins = df[:,1]

    println("origins type")
    println(typeof(origins))
    
    #println("print for i in origins")
    #for i in origins
    #    println(i)
    #end

    facilities = Int64[]
    for i in df[1,2:end]
      push!(facilities,trunc(Int, i))
    end

    println("facilities type")
    println(typeof(facilities))
    
    println("facilities j")
    for j in 1:length(facilities)
        println(j)
    end
    
    println("facilities[j]")
    for j in 1:length(facilities)
        println(facilities[j])
    end
    
    #m = Model(with_optimizer(CPLEX.Optimizer))
    #output says threads were changed, but I do not see a difference on the resource monitor
    #m = Model(with_optimizer(Cbc.Optimizer, threads = 14))
    #change the limit to 
    m = Model(with_optimizer(Cbc.Optimizer, threads = 2, seconds = 68400))

    # Facility locations
    #@variable(m, 0 <= s[1:numLocation] <= 1)
    #@variable(m, 0 <= x[1:length(facilities)] <= 1)
    #binary variable
    @variable(m, x[1:length(facilities)], binary=true)

    #println("print Facility location var")
    #println(x)

    # Aux. variable: x_a,i = 1 if the closest facility to a is at i
    #@variable(m, 0 <= x[1:numLocation,1:numCustomer] <= 1)
    #@variable(m, 0 <= y[origins,1:length(facilities)] <= 1)
    #binary variable
    @variable(m, y[origins,1:length(facilities)], binary=true)

    #println("print origin facility var")
    #println(y)
    
    
    #println("test")
    #for j in 1:length(facilities)
    #    for i in origins
    #       println(OD_dict[i][j])
    #    end
    #end

    # Objective: min distance
    #@objective(m, Min, sum(abs(customerLocations[a]-i)*x[i,a] for a = 1:numCustomer, i = 1:numLocation) )

    @objective(m, Min, sum(OD_dict[i][j]*y[i,j] for i in origins, j = 1:length(facilities)) )

    # Constraints


    # Subject to must allocate all facilities
    @constraint(m, sum(x[i] for i=1:length(facilities)) == numFacility )


    for i in origins
        # Subject to linking x with s
        for j in 1:length(facilities)
            @constraint(m, y[i,j] <= x[j])
        end

        # Subject to one of x must be 1
        @constraint(m, sum(y[i,j] for j=1:length(facilities)) == 1 )
    end
    
    
    # capacity constraints
    for j in 1:length(facilities)
        @constraint(m, sum(y[i,j] * origins_pop_dict[i] for i in origins) <= facilities_cap_dict[facilities[j]])
    end


    JuMP.optimize!(m)

    println("Objective value is: ", JuMP.objective_value(m))

    #println("Objective bound is: ", JuMP.objective_bound(m))


    println("print array values")
    println(value.(x))
    println("print array length")
    println(length(value.(x)))

    result_array = value.(x)

    selected_facilities = []

    for i=1:length(result_array)
       if result_array[i] == 1
           push!(selected_facilities,column_headers[i])
       end
    end

    println("print selected_facilities")
    println(selected_facilities)

    #save selected_facilities array to file
    #C:\Users\gost_\Desktop\lima\data\OD_distance
    #writedlm("C:\\Users\\gost_\\Desktop\\lima\data\\OD_distance\\selected_facilities_array", selected_facilities)
    #writedlm("H:\\lima_optimality\\examples_testing\\OD2\\selected_facilities_array", selected_facilities)
    #writedlm("C:\\Temp\\lima_OD_distance_output\\selected_facilities_array_lima_distance_weighted_12hr_v2_binary_vars", selected_facilities)

    #println("finished writing selected_facilities_array to file")

    if termination_status(m) == MOI.OPTIMAL
        optimal_solution = value.(x)
        optimal_objective = objective_value(m)
    elseif termination_status(m) == MOI.TIME_LIMIT && has_values(model)
        suboptimal_solution = value.(x)
        suboptimal_objective = objective_value(m)
    else
        error("The model was not solved correctly.")
    end

    return selected_facilities

end

Cap_pMedian (generic function with 1 method)

### The pMedian function takes the number of facilities to place as the first input. For the second input it takes in the OD matrix as a csv file. For the third input it takes in a dictionary of the origins with their populations. For the forth input it takes a dictionary of facilities with their capacity.

### You may import a facilities_cap_series from csv, but in this case we will generate our own test data

In [17]:
# materialize a csv file as a DataFrame
df = CSV.File("../../../../lima_optimization_output/saved_OD.csv") |> DataFrame!

,Column1,6048,2048,6691,4154,4198,4647,4233,3914
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,6147,1968.66,1020.72,363.677,819.75,1322.0,1578.08,1803.25,806.455
2,2052,525.817,558.843,1633.65,1591.88,274.594,839.743,1166.55,1578.63
3,3,1330.77,448.775,1230.2,1380.17,873.074,1498.69,1962.52,1307.03
4,6154,153.55,1138.25,1658.31,1700.64,853.999,1304.18,723.313,1988.98
5,6162,202.318,907.368,1799.38,1663.94,623.119,1074.26,845.305,1927.15
6,4115,802.824,1190.08,1769.8,1064.24,763.704,477.901,500.301,1352.59
7,6165,953.191,391.808,1494.09,1121.14,280.176,730.055,1171.03,1098.5
8,21,697.559,1618.36,2043.28,1814.87,1190.01,1528.51,820.985,2103.22
9,4125,799.148,1715.41,2094.9,1389.34,1288.38,1016.71,236.478,1677.68


In [18]:
facilities = []
for i in df[1,2:end]
  push!(facilities,trunc(Int, i))
end

In [19]:
facilities

17-element Array{Any,1}:
 1968
 1020
  363
  819
 1322
 1578
 1803
  806
 2049
 1517
 1879
  803
 1542
 1657
  583
 1736
 1584

In [102]:
# create a dictionary of facilities and assign them a random capacity between 20k and 50k
facilities_cap_dict = Dict()
for i in 1:size(facilities,1)
    facilities_cap_dict[facilities[i,1]] = rand(200000:500000)
end

In [103]:
facilities_cap_dict

Dict{Any,Any} with 17 entries:
  1879 => 352151
  1578 => 444537
  1736 => 221651
  1584 => 293965
  2049 => 458020
  1542 => 292101
  803  => 486850
  583  => 289452
  819  => 382890
  363  => 439585
  1803 => 406534
  1322 => 243748
  1968 => 447991
  806  => 338052
  1517 => 443935
  1657 => 320002
  1020 => 301991

In [104]:
facilities_cap_dict[806]

338052

#### import an origins_pop_series from csv


In [105]:
# materialize a csv file as a DataFrame
origins_pop_series = CSV.File("../../../../lima_optimization_output/origins_w_demands_series_no_dupl.csv") |> DataFrame!

,NN,pop
,Int64,Float64
1,3,1458.0
2,21,2232.0
3,32,2041.0
4,82,1508.0
5,84,1610.0
6,99,1295.0
7,106,1216.0
8,114,824.0
9,124,440.0


In [106]:
origins_pop_dict = Dict()
for i in 1:size(origins_pop_series,1)
    origins_pop_dict[origins_pop_series[i,1]] = origins_pop_series[i,2]
end

In [107]:
origins_pop_dict

Dict{Any,Any} with 678 entries:
  3847 => 847.0
  1090 => 1371.0
  4130 => 130.0
  1333 => 141.0
  2812 => 1843.0
  3485 => 2131.0
  2564 => 1535.0
  5162 => 436.0
  5476 => 2520.0
  1662 => 1474.0
  1461 => 1715.0
  4223 => 1625.0
  1124 => 1705.0
  3181 => 1409.0
  6440 => 1525.0
  2835 => 2374.0
  1845 => 2921.0
  5784 => 2197.0
  563  => 4970.0
  2202 => 160.0
  3213 => 2394.0
  2354 => 2584.0
  671  => 110.0
  3126 => 4035.0
  5743 => 1392.0
  ⋮    => ⋮

In [108]:
origins_pop_dict[3]

1458.0

In [112]:
selected_facilities = Cap_pMedian(4,"../../../../lima_optimization_output/saved_OD.csv", origins_pop_dict, facilities_cap_dict)

numFacility
4
origins type
Array{Int64,1}
facilities type
Array{Int64,1}
facilities j
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
facilities[j]
1968
1020
363
819
1322
1578
1803
806
2049
1517
1879
803
1542
1657
583
1736
1584
Objective value is: 312049.36805564194
print array values
[0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
print array length
17
print selected_facilities
Any["2048", "4154", "3409", "6107"]
Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -threads 2 -seconds 68400 -solve -quit (default strategy 1)
threads was changed from 0 to 2
seconds was changed from 1e+100 to 68400
Continuous objective value is 312049 - 0.73 seconds
Cgl0004I processed model has 12222 rows, 11543 columns (11543 integer (11543 of which binary)) and 46121 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 6.17284e-14
Cbc0038I Solution found of 312049
Cbc0038I Before mini branch and bound, 11543 i

4-element Array{Any,1}:
 "2048"
 "4154"
 "3409"
 "6107"

In [43]:
selected_facilities

4-element Array{Any,1}:
 "2048"
 "4154"
 "3409"
 "6107"

In [45]:
#write-out selected_facilities
writedlm("../../../../lima_optimization_output/selected_facilities_file_from_julia",selected_facilities)